# Librerias de Kaggle

In [14]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
from keras.models import Sequential
#Import from keras_preprocessing not from keras.preprocessing
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
#import pandas as pd
#import numpy as np

from keras_preprocessing.image import ImageDataGenerator
import pandas as pd

import matplotlib.pyplot as plt

Using TensorFlow backend.


Cargando la data desde el archivo csv


In [2]:
labels=pd.read_csv(r"data/train_labels.csv")
path = "data/"

In [3]:
def append_ext(fn):
    return fn+".jpg"

traindf=pd.read_csv(path+'train_labels.csv',dtype=str)

traindf["Id"]=traindf["Id"].apply(append_ext)

Utilizamos generadores de keras para trabajar con las imagenes

In [4]:
datagen=ImageDataGenerator(rescale=1./255.,validation_split=0.25)

In [5]:
train_generator=datagen.flow_from_dataframe(
dataframe=traindf,
directory=path+"train_images/",
x_col="Id",
y_col="Expected",
subset="training",
batch_size=32,
seed=30,
shuffle=True,
class_mode="categorical",
target_size=(128,128))

Found 6909 validated image filenames belonging to 10 classes.


In [6]:
valid_generator=datagen.flow_from_dataframe(
dataframe=traindf,
directory=path+"train_images/",
x_col="Id",
y_col="Expected",
subset="validation",
batch_size=32,
seed=30,
shuffle=True,
class_mode="categorical",
target_size=(128,128))

Found 2303 validated image filenames belonging to 10 classes.


Primer modelo de CNN

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(128,128,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
model.compile(optimizers.rmsprop(lr=0.0001, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])

In [8]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

Segundo modelo CNN

In [15]:
test_files_df=pd.DataFrame()
test_files_df['file']=os.listdir(path+'test_images')
#print("Loaded test files list")

In [16]:
test_generator=ImageDataGenerator(rescale=1./255.).flow_from_dataframe(
                    dataframe=test_files_df,
                    directory=path+"test_images",
                    x_col="file",
                    y_col=None,
                    has_ext=True,
                    class_mode=None,
                    batch_size=32,
                    seed=30,
                    shuffle=False,
                    target_size=(128,128))  

Found 1023 validated image filenames.


Predicción de modelo 1 (Model) 

In [ ]:
y_pred = model.predict_generator(test_generator)
y_pred = np.argmax(y_pred, axis=1)

Guardando en csv

In [ ]:
# Create submission df
submission_df = pd.DataFrame({
    'Id':test_generator.filenames,
    'Expected':y_pred })
# Filename is id, remove extension .tif
submission_df['Id'] = submission_df['Id'].apply(lambda x: x.split('.')[0])
#print(f"Submission dataframe created. Rows:{len(submission_df.values)}")

In [ ]:
submission_df

In [ ]:
#train_generator.class_indices
for key in train_generator.class_indices:
    submission_df['Expected'] = submission_df['Expected'].replace(train_generator.class_indices[key],key)

In [ ]:
submission_df

In [ ]:
submission_df.to_csv('6-submission.csv', index=False)

In [34]:
# Configure the CNN (Convolutional Neural Network).
classifier = Sequential()
'''
# Convolution - extracting appropriate features from the input image.
# Non-Linearity (RELU) - replacing all negative pixel values in feature map by zero.
classifier.add(Conv2D(32, (3, 3), input_shape=(128, 128, 3),
               activation='relu'))
# Pooling: reduces dimensionality of the feature maps but keeps the most important information.
classifier.add(MaxPooling2D(pool_size=(2, 2)))
# Adding a second convolutional layer and flattening in order to arrange 3D volumes into a 1D vector.
classifier.add(Conv2D(32, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Flatten())
# Fully connected layers: ensures connections to all activations in the previous layer.
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dense(units=10, activation='softmax'))
'''

classifier.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
classifier.add(MaxPooling2D((2, 2)))
classifier.add(Conv2D(64, (3, 3), activation='relu'))
classifier.add(MaxPooling2D((2, 2)))
classifier.add(Conv2D(64, (3, 3), activation='relu'))
classifier.add(MaxPooling2D((2, 2)))
classifier.add(Flatten()) 
classifier.add(Dense(64, activation='relu'))
classifier.add(Dense(10, activation='softmax'))

# Compile the CNN and train the classifier..
classifier.compile(optimizer='adam', loss='binary_crossentropy',
                   metrics=['accuracy'])
#from keras.preprocessing.image import ImageDataGenerator

history=classifier.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=5
)

Epoch 1/5
215/215 [==============================] - 139s 645ms/step - loss: 0.2336 - accuracy: 0.9138 - val_loss: 0.1648 - val_accuracy: 0.9260
Epoch 2/5
215/215 [==============================] - 128s 596ms/step - loss: 0.1659 - accuracy: 0.9351 - val_loss: 0.1719 - val_accuracy: 0.9350
Epoch 3/5
215/215 [==============================] - 140s 651ms/step - loss: 0.1357 - accuracy: 0.9463 - val_loss: 0.2044 - val_accuracy: 0.9409
Epoch 4/5
215/215 [==============================] - 146s 677ms/step - loss: 0.1096 - accuracy: 0.9573 - val_loss: 0.1387 - val_accuracy: 0.9487
Epoch 5/5
215/215 [==============================] - 138s 641ms/step - loss: 0.0834 - accuracy: 0.9680 - val_loss: 0.1074 - val_accuracy: 0.9476


Predicción con segundo modelo

In [24]:
y_pred = classifier.predict_generator(test_generator)
#y_pred = np.argmax(y_pred, axis=1)

In [27]:
y_pred.shape

(1023, 10)

Guardando resultados de predicción para segundo modelo (classifier)

In [ ]:
# Create submission df
submission_df2 = pd.DataFrame({
    'Id':test_generator.filenames,
    'Expected':y_pred })
# Filename is id, remove extension .tif
submission_df2['Id'] = submission_df2['Id'].apply(lambda x: x.split('.')[0])
#print(f"Submission dataframe created. Rows:{len(submission_df.values)}")

In [ ]:
for key in train_generator.class_indices:
    submission_df2['Expected'] = submission_df2['Expected'].replace(train_generator.class_indices[key],key)
    
submission_df2.to_csv('5-submission.csv', index=False)
submission_df2

In [ ]:
model_2 = Sequential()
model_2.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model_2.add(MaxPooling2D((2, 2)))
model_2.add(Conv2D(64, (3, 3), activation='relu'))
model_2.add(MaxPooling2D((2, 2)))
model_2.add(Conv2D(64, (3, 3), activation='relu'))

model_2.add(Flatten()) 
model_2.add(Dense(64, activation='relu'))
model_2.add(Dense(10, activation='softmax'))

In [ ]:
model_2.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model_2.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=12)

In [ ]:
y_pred_2 = model_2.predict_generator(test_generator)
y_pred_2 = np.argmax(y_pred_2, axis=1)

In [ ]:
# Create submission df
submission_df3 = pd.DataFrame({
    'Id':test_generator.filenames,
    'Expected':y_pred_2 })
# Filename is id, remove extension .tif
submission_df3['Id'] = submission_df3['Id'].apply(lambda x: x.split('.')[0])
#print(f"Submission dataframe created. Rows:{len(submission_df.values)}")

for key in train_generator.class_indices:
    submission_df3['Expected'] = submission_df3['Expected'].replace(train_generator.class_indices[key],key)
    
submission_df3.to_csv('7-submission.csv', index=False)
submission_df3